In [1]:
import pandas as pd
df = pd.read_csv('C:\\Users\\moham\\Google Drive\\NU\\Graduation Project\\Smart-Customer-Care\\Scrapper\\Twitter Scrapper\\Scrapping results after cleanning.csv', )
#df

,Unnamed: 0,Tweet,Location,year,month,day
0,0,اللي بينا مايتقدر عمرو_دياب فودافون httpstco...,"Cairo, Egypt",2022,4,4
1,1,ما بعد فوق قمة الاستهتار والاهمال واللا مبالاة...,Egypt,2022,4,4
2,2,عاجل فضيحة فودافون تتسبب في أزمة بعد تسريب_بيا...,dokki,2022,4,4
3,3,10700 وحدة 3270105200846858 فودافون,مصر,2022,4,3
4,4,فودافون 8584575262021929698 250 وحده 85839...,مصر,2022,4,3
...,...,...,...,...,...,...
4786,4786,كسم we على كسم فودافون كفرتوا الواحد,"Cairo, Egypt",2022,3,27
4787,4787,شنط رمضان 11 رمضان بلا محتاج للتبرع فودافون...,tanta,2022,3,27
4788,4788,الخط فودافون و الراوتر فودافون و لما شبكة زفت ...,cairo,2022,3,27
4789,4789,شركه فودافون شركه حراميه,Banha,2022,3,27


In [2]:
from camel_tools.utils.charsets import AR_LETTERS_CHARSET
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.sentiment import SentimentAnalyzer

    
class Morphological_Analysis():
    
    
    def __init__(self, DF, COL):
        self.df = DF
        self.col = COL
        

    def del_nonAR(self):
        for rows in self.df[self.col]:
            for char in rows:
                if char not in AR_LETTERS_CHARSET and char != ' ' :
                    self.df[self.col] = self.df[self.col].str.replace(char, '')
        return df
                    
    def get_lex(self):
        
        db = MorphologyDB.builtin_db('calima-egy-r13')
        analyzer = Analyzer(db)
        
        for rows in df[col]:
            sentence = simple_word_tokenize(rows)
            lex_sentence = ''
            for words in sentence:
                try:
                    lex_sentence += analyzer.analyze(words)[0]['root'] + ' '
                    lex_sentence = lex_sentence.replace('.', '')
                except:
                    lex_sentence += ' ' + words + ' '
            print(lex_sentence)
            df[col] = df[col].str.replace(rows, lex_sentence)
        return df
    
    #def del_stopwords(df, col):
            
class Sentiment_Analysis:
    
    def __init__(self, DF, COL):
        self.df = DF
        self.col = CO
    
    def get_sentiment(self):
        sa = SentimentAnalyzer.pretrained()
        sentences = []
        for rows in self.df[self.col]:
            sentences.append(rows)
        sentiments = sa.predict(sentences)
        self.df['Sentiment'] = sentiments
        return df

Removing Non-Arabic Characters

In [ ]:
Morph = Morphological_Analysis(df, 'Tweet')

In [ ]:
Morph.del_nonAR()

Get Words' Root

In [ ]:
#Morph.get_lex(df, 'Tweet')
df.fillna(value=" ", inplace=True)
#df

Get Sentiments

In [ ]:
Sent = Sentiment_Analysis
Sent.get_sentiment(df, 'Tweet')

In [ ]:
df.to_csv('Tweets.csv', index=False)

In [ ]:
#df

In [ ]:
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

mled = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mled, 'pos')

tagger.tag('و عملية برقم تم استرداد المبلغ الي محفظة فودا'.split())

In [ ]:
# no. of retweets
api.get_status(id).retweet_count